# Deliverable 1: Retrieve Weather Data

1. Create a folder called Weather_Database.
2. Create a new Jupyter Notebook file to retrieve the weather data, and name it Weather_Database.ipynb.
3. Create a new set of 2,000 random latitudes and longitudes.
4. Get the nearest city using the citipy module.
5. Perform an API call with the OpenWeatherMap.
6. Retrieve the following information from the API call:
    a. Latitude and longitude
    b. Maximum temperature
    c. Percent humidity
    d. Percent cloudiness
    e. Wind speed
    f. Weather description (for example, clouds, fog, light rain, clear sky)
7. Add the data to a new DataFrame.
8. Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder.

In [8]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
import os
import time

from citipy import citipy

# Import the API keys.
from config import weather_api_key

In [9]:
# 3. Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [10]:
# Add the latitudes and longitudes to a list.
coord = list(lat_lngs)

In [11]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coord:
    print(coordinate[0], coordinate[1])

-47.34493932274701 -21.450009358916844
-66.26790263823635 -102.57889256517107
-37.446772588837156 67.01316138014147
-65.1098258255065 -87.95144429594797
-80.11266390877688 -67.89177643239063
66.83086415292448 -21.441293118123298
18.248757407025764 -126.63393049185075
38.46371444098932 34.60697299601492
-16.867313909715833 -176.75203727948508
-47.022166701369166 -4.819244285445791
-23.620478290268494 149.73232427987892
26.910661406616157 -58.427258708090264
-84.43722764482736 20.578685142382795
-30.97370990833275 3.5262752201238925
28.056412883414083 -55.76728267085667
-78.83545883575309 87.10180996048916
-56.60551103155986 28.109646079873784
-68.72166833211703 166.90232567851268
-55.804243905197 -105.63806313139112
84.9010453318549 74.57455943824746
70.77182908898357 -3.204123799218081
40.00455051114321 100.14187641749277
-41.415016680880655 -54.68057996607915
-9.906118142248516 -150.91543803271634
72.97151521883208 5.673213022900143
65.63365589612599 30.488173594317573
-26.72566377195

2.5967770153476266 -57.89306420923853
57.2416289019541 37.77529034874317
-55.89145775691026 -125.49874575151449
-21.54035378372656 148.23157789424135
-25.18282482442109 -90.57349927806094
9.454443850530694 99.1627618959069
-51.83012184480654 165.89708346597308
-0.6509029725549738 -68.82104450505867
60.40215211161265 -107.78698211309418
45.51905327150402 -18.213609514990964
67.29080684972129 -6.266571407822852
28.68314806182603 -20.63992822888636
-39.019291560052146 -175.69926065100998
-10.861147549209164 16.80363089386563
39.5536037244303 -38.26888417045228
-38.48647705247527 -75.28623060161677
59.42924433407569 88.58141137022977
-43.74071898739528 -31.60476947488371
-9.972009145161266 146.59198683921204
-43.86325460404879 23.723420983587573
-89.36961268551758 164.7696046563894
-26.602695561816496 -155.97585401948524
-33.76017178877885 -160.62305688031105
81.44914916165445 -7.217452094452796
-41.265223022829815 2.8722641465673746
33.523859550111794 39.83118627233978
85.74112437105268 1

In [12]:
# 4. Get the nearest city using the citipy module and create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coord:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

746

In [13]:
# 5. Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [14]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# 6. Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
     
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_descr = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_descr,
                          "Country": city_country})
        
        
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | arraial do cabo
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | souillac
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | skagastrond
City not found. Skipping...
Processing Record 6 of Set 1 | guerrero negro
Processing Record 7 of Set 1 | nevsehir
Processing Record 8 of Set 1 | hihifo
City not found. Skipping...
Processing Record 9 of Set 1 | cape town
Processing Record 10 of Set 1 | rockhampton
Processing Record 11 of Set 1 | saint george
Processing Record 12 of Set 1 | bredasdorp
Processing Record 13 of Set 1 | luderitz
Processing Record 14 of Set 1 | busselton
Processing Record 15 of Set 1 | port elizabeth
Processing Record 16 of Set 1 | bluff
Processing Record 17 of Set 1 | castro
Processing Record 18 of Set 1 | dikson
Processing Record 19 of Set 1 | klaksvik
Processing Record 20 of Set 1 | zhangye
Processing Record 21 of Set 1 | mar del plata
P

Processing Record 38 of Set 4 | tazovskiy
Processing Record 39 of Set 4 | ndungu
Processing Record 40 of Set 4 | aquidauana
Processing Record 41 of Set 4 | shenjiamen
Processing Record 42 of Set 4 | sola
Processing Record 43 of Set 4 | lavrentiya
Processing Record 44 of Set 4 | labuhan
Processing Record 45 of Set 4 | vila
Processing Record 46 of Set 4 | sentyabrskiy
City not found. Skipping...
Processing Record 47 of Set 4 | gimbi
Processing Record 48 of Set 4 | tsihombe
City not found. Skipping...
Processing Record 49 of Set 4 | cocoa beach
Processing Record 50 of Set 4 | chake chake
Processing Record 1 of Set 5 | tasiilaq
Processing Record 2 of Set 5 | mogadishu
Processing Record 3 of Set 5 | florence
Processing Record 4 of Set 5 | vanimo
Processing Record 5 of Set 5 | progreso
Processing Record 6 of Set 5 | yangjiang
Processing Record 7 of Set 5 | abu samrah
Processing Record 8 of Set 5 | saldanha
Processing Record 9 of Set 5 | phan rang
City not found. Skipping...
Processing Record

Processing Record 26 of Set 8 | tambopata
Processing Record 27 of Set 8 | lima
Processing Record 28 of Set 8 | buariki
City not found. Skipping...
Processing Record 29 of Set 8 | utiroa
City not found. Skipping...
Processing Record 30 of Set 8 | bartica
Processing Record 31 of Set 8 | marystown
Processing Record 32 of Set 8 | takoradi
Processing Record 33 of Set 8 | padang
Processing Record 34 of Set 8 | coquimbo
Processing Record 35 of Set 8 | riberalta
Processing Record 36 of Set 8 | la ronge
Processing Record 37 of Set 8 | mier
Processing Record 38 of Set 8 | yanahuanca
Processing Record 39 of Set 8 | hualmay
Processing Record 40 of Set 8 | tapaua
City not found. Skipping...
Processing Record 41 of Set 8 | severo-kurilsk
Processing Record 42 of Set 8 | denpasar
Processing Record 43 of Set 8 | marawi
Processing Record 44 of Set 8 | divinopolis
Processing Record 45 of Set 8 | bac lieu
Processing Record 46 of Set 8 | northam
Processing Record 47 of Set 8 | port blair
Processing Record 

Processing Record 14 of Set 12 | hare bay
Processing Record 15 of Set 12 | bilibino
Processing Record 16 of Set 12 | angoche
Processing Record 17 of Set 12 | fort nelson
Processing Record 18 of Set 12 | conceicao do araguaia
Processing Record 19 of Set 12 | karratha
Processing Record 20 of Set 12 | emba
Processing Record 21 of Set 12 | jacksonville
Processing Record 22 of Set 12 | yaan
Processing Record 23 of Set 12 | muzambinho
Processing Record 24 of Set 12 | ilulissat
Processing Record 25 of Set 12 | tibu
Processing Record 26 of Set 12 | tambacounda
Processing Record 27 of Set 12 | umzimvubu
City not found. Skipping...
Processing Record 28 of Set 12 | tomigusuku
Processing Record 29 of Set 12 | karasjok
Processing Record 30 of Set 12 | pangai
Processing Record 31 of Set 12 | peniche
Processing Record 32 of Set 12 | belmonte
Processing Record 33 of Set 12 | mayo
Processing Record 34 of Set 12 | havre-saint-pierre
Processing Record 35 of Set 12 | tandil
Processing Record 36 of Set 12 

In [15]:
print ("The length of the Dictionary is", len(city_data))

The length of the Dictionary is 688


In [16]:
# 7. Convert the array of dictionaries to a Pandas dataframe.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,Arraial Do Cabo,-22.9661,-42.0278,69.66,73,75,6.91,broken clouds,BR
1,Punta Arenas,-53.1500,-70.9167,37.51,81,20,23.02,few clouds,CL
2,Souillac,-20.5167,57.5167,70.18,73,20,8.05,few clouds,MU
3,Ushuaia,-54.8000,-68.3000,29.86,100,100,2.30,snow,AR
4,Guerrero Negro,27.9769,-114.0611,73.38,51,0,19.10,clear sky,MX
5,Nevsehir,38.9167,34.6667,65.41,56,0,5.75,clear sky,TR
6,Cape Town,-33.9258,18.4232,56.12,75,0,10.36,clear sky,ZA
7,Rockhampton,-23.3833,150.5000,55.29,88,100,1.14,overcast clouds,AU
8,Saint George,37.1041,-113.5841,81.95,7,0,19.57,clear sky,US
9,Bredasdorp,-34.5322,20.0403,50.13,82,13,4.00,few clouds,ZA


In [20]:
# Reorder the columns in the order you want them to appear.
new_column_order = ["City", "Country","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

# Assign city_data df the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Arraial Do Cabo,BR,-22.9661,-42.0278,69.66,73,75,6.91,broken clouds
1,Punta Arenas,CL,-53.1500,-70.9167,37.51,81,20,23.02,few clouds
2,Souillac,MU,-20.5167,57.5167,70.18,73,20,8.05,few clouds
3,Ushuaia,AR,-54.8000,-68.3000,29.86,100,100,2.30,snow
4,Guerrero Negro,MX,27.9769,-114.0611,73.38,51,0,19.10,clear sky
5,Nevsehir,TR,38.9167,34.6667,65.41,56,0,5.75,clear sky
6,Cape Town,ZA,-33.9258,18.4232,56.12,75,0,10.36,clear sky
7,Rockhampton,AU,-23.3833,150.5000,55.29,88,100,1.14,overcast clouds
8,Saint George,US,37.1041,-113.5841,81.95,7,0,19.57,clear sky
9,Bredasdorp,ZA,-34.5322,20.0403,50.13,82,13,4.00,few clouds


In [23]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")